In [1]:
from datetime import datetime
import struct
import numpy as np

In [2]:
# We'll use the current time as a timestamp
now = datetime.utcnow()
now

datetime.datetime(2020, 8, 13, 5, 56, 21, 952464)

In [3]:
# For our sample strike, I'm going to use the lat/lon of Huntsville, Alabama,
# and assume the intensity is 12.8 kAmps
lat = 34.73
lon = -86.585
kamps = 12.8

In [4]:
# The strike message is 24 bytes.  We could use a
# bytearray, but NumPy makes it easier to calculate
# the checksum.
msg = np.zeros(shape=(24,), dtype=np.uint8)

In [5]:
# The "minutes" field is really combined hours and minutes.
minutes = now.hour * 60 + now.minute

# The "milliseconds" is likewise the seconds and milliseconds.
# (Python datetime gives us microseconds, but the output is
# only precise to milliseconds.)
msec = now.second * 1000 + now.microsecond // 1000

year = now.year
month = now.month
day = now.day

In [6]:
# Day of the month is split into two nybbles
day_hi = (day >> 4) & 0xF
day_lo = day & 0xF

# Some other fields are split into two bytes
year_lo = year & 0xFF
year_hi = (year >> 8) & 0xF
minutes_lo = minutes & 0xFF
minutes_hi = (minutes >> 8) & 0xFF
msec_lo = msec & 0xFF
msec_hi = (msec >> 8) & 0xFF

In [7]:
# The first byte gives the message type:
# 0 for cloud-to-ground
# 2 for network timing
# 3 for cloud-to-cloud
msg[0] = 0

# There's really no rhyme or reason as to how the timestamp
# is encoded in the next seven bytes.  Some fields are done
# little-endian, some big-endian, some divided at nybbles,
# and the year is split between two bytes that are not
# consecutive.
msg[1] = minutes_hi
msg[2] = minutes_lo
msg[3] = msec_lo
msg[4] = msec_hi
msg[5] = year_lo
msg[6] = (month << 4) | day_hi
msg[7] = (day_lo << 4) | year_hi

In [8]:
# Lat/lon are scaled by 10 million.  Power is in hectoamps.
ilat = int(lat * 1e7)
ilon = int(lon * 1e7)
hectoamps = round(kamps * 10)

In [9]:
# Pack this into the message buffer, starting from byte 8.  (We already
# populated the first 8 bytes above.)  This doesn't set the GDOP or error
# ellipse fields, so those will be left as zero.
struct.pack_into('<lBhl', msg, 8, ilon, 0, hectoamps, ilat)

In [10]:
# Now compute the checksum, which goes in the last byte of the message.
checksum = 256 - int(np.sum(msg, dtype=np.uint8))
msg[23] = checksum

In [11]:
msg

array([  0,   1, 100, 192,  85, 228, 128, 215, 113,  45, 100, 204,   0,
       128,   0, 159,  96, 179,  20,   0,   0,   0,   0,  55], dtype=uint8)

In [12]:
with open('/tmp/toa.dat', 'wb') as of:
    of.write(msg.tobytes())